
## Import 

In [1]:
import os, sys, pickle

import requests

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
import plotly.express as px

from sklearn.model_selection import * 
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.impute import * 
from sklearn.preprocessing import * 
from sklearn.ensemble import * 
from  xgboost import XGBClassifier



import shap

## Load data

In [2]:
url = "https://gist.githubusercontent.com/AlexandreGazagnes/9018022652ba0933dd39c9df8a600292/raw/0845ef4c2df4806bb05c8c7423dc75d93e37400f/titanic_train_raw_csv"

In [3]:
df = pd.read_csv (url)

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Select 

In [5]:
cols = ["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
df = df.loc[:, cols]
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [6]:
"""categorical_features = ["Embarked", "Sex"]
for cat in categorical_features : 
    df[cat] = pd.Categorical(df[cat])

df.head()
"""

'categorical_features = ["Embarked", "Sex"]\nfor cat in categorical_features : \n    df[cat] = pd.Categorical(df[cat])\n\ndf.head()\n'

In [7]:
df = pd.get_dummies(df)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Age         714 non-null    float64
 3   SibSp       891 non-null    int64  
 4   Parch       891 non-null    int64  
 5   Fare        891 non-null    float64
 6   Sex_female  891 non-null    uint8  
 7   Sex_male    891 non-null    uint8  
 8   Embarked_C  891 non-null    uint8  
 9   Embarked_Q  891 non-null    uint8  
 10  Embarked_S  891 non-null    uint8  
dtypes: float64(2), int64(4), uint8(5)
memory usage: 46.2 KB


## Prepare

In [9]:
X = df.drop(columns="Survived")
y = df.Survived

In [10]:
X_train, X_test, y_train, y_test  = train_test_split(X, y)

## Model

In [11]:
"""

categorical_features = ["Embarked", "Sex"]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder='passthrough'
)

"""

'\n\ncategorical_features = ["Embarked", "Sex"]\ncategorical_transformer = OneHotEncoder(handle_unknown="ignore")\n\npreprocessor = ColumnTransformer(\n    transformers=[\n        ("cat", categorical_transformer, categorical_features),\n    ],\n    remainder=\'passthrough\'\n)\n\n'

In [12]:
imputer = KNNImputer().fit(X_train)

X_train = pd.DataFrame(imputer.transform(X_train), columns=X.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X.columns)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      668 non-null    float64
 1   Age         668 non-null    float64
 2   SibSp       668 non-null    float64
 3   Parch       668 non-null    float64
 4   Fare        668 non-null    float64
 5   Sex_female  668 non-null    float64
 6   Sex_male    668 non-null    float64
 7   Embarked_C  668 non-null    float64
 8   Embarked_Q  668 non-null    float64
 9   Embarked_S  668 non-null    float64
dtypes: float64(10)
memory usage: 52.3 KB


In [13]:
"""

pipe = Pipeline([

    ("imputer",      KNNImputer() ),
    # ("scaler",       "passthrough"),
    ("estimator",    RandomForestClassifier()),
    ])
"""

'\n\npipe = Pipeline([\n\n    ("imputer",      KNNImputer() ),\n    # ("scaler",       "passthrough"),\n    ("estimator",    RandomForestClassifier()),\n    ])\n'

In [14]:
grid = GridSearchCV(XGBClassifier(), {}, cv=5, return_train_score=True, n_jobs=-1)

## Train

In [69]:
grid.fit(X_train, y_train)

## Test

In [ ]:
res = pd.DataFrame(grid.cv_results_)
res = res.loc[:, [i for i in res.columns if "split" not in i ]]
res

In [ ]:
grid.score(X_test, y_test)

In [ ]:
model = grid.best_estimator_
model

## Explain

In [ ]:
estimator = model
feat = pd.Series(estimator.feature_importances_, index=X_test.columns)# .sort_values(ascending=False)
px.bar(feat)

In [ ]:
estimator = model
feat = pd.Series(estimator.feature_importances_, index=X_test.columns).sort_values(ascending=False)
px.bar(feat)

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer.shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar",show=True)

In [ ]:
"""
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.savefig('grafic.png')
"""

In [ ]:
"""
shap.plots.waterfall(shap_values[0])
"""

In [ ]:
"""

explainer = shap.TreeExplainer(model, X_train)
shap_values = explainer(X_train)
shap.plots.waterfall(shap_values[0])
"""

In [ ]:
shap.plots.waterfall(shap_values[0])

## Save

In [ ]:
with open("./models/model.pk", "wb") as f : 
    pk = pickle.dumps(grid)
    f.write(pk)